# Fundamentals of Data Analysis Winter 2023 Tasks

**Author: Nur Bujang**

tasks.ipynb
***

## Task 1 : Collatz Conjecture
> The Collatz conjecture is a famous unsolved problem in mathe matics. The problem is to prove that if you start with any positive integer x and  repeatedly apply the function f ( x ) below, you al ways get stuck in the repeating sequence 1, 4, 2, 1, 4, 2, . . .

$$ f(x) =  \Biggl\{ $$

For example, starting with the value 10, which is an even number, we divide it by 2 to get 5. Then 5 is an odd number so, we multiply by 3 and add 1 to get 16. Then we repeatedly divide by 2 to get 8, 4, 2, 1. Once we are at 1, we go back to 4 and get stuck in the repeating sequence 4, 2, 1 as we suspected.

### Task Description:
*The task is to verify, using Python, that the Collatz conjecture is true for the first 10000 positive integers.*


In [ ]:
def f(x):
    # If x is even, divide it by two.
    if x % 2 ==0:
        return x // 2
    else:
        return (3 * x) + 1

In [ ]:
def collatz(x):
    while x != 1:
        print(x, end=', ')
        x = f(x)
    print(x)

x = f(x)
Then print out new x value

In [ ]:
collatz(10000)

next is to loop the collatz

***

## End